<a href="https://colab.research.google.com/github/karunaprakash062/Major_Project/blob/main/featureselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('/content/drive/MyDrive/NSL-KDD/NSL_KDD_Data.csv')

In [4]:
data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,d_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [5]:
classes_to_remove = ['spy', 'warezclient']

In [6]:
train_data = data[~data['class'].isin(classes_to_remove)]

In [7]:
train_data.reset_index(drop=True, inplace=True)

In [8]:
removed_feature = train_data.pop('num_outbound_cmds')

In [9]:
unique_train_classes = set(train_data['class'])

In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_data['service'] = encoder.fit_transform(train_data['service'])
train_data['protocol_type'] = encoder.fit_transform(train_data['protocol_type'])
train_data['flag'] = encoder.fit_transform(train_data['flag'])

<ipython-input-10-f793b94e1e9b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['service'] = encoder.fit_transform(train_data['service'])
<ipython-input-10-f793b94e1e9b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['protocol_type'] = encoder.fit_transform(train_data['protocol_type'])
<ipython-input-10-f793b94e1e9b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [11]:
final_train_data = train_data.copy()
target = train_data.pop('class')

In [12]:
target = encoder.fit_transform(target)

In [13]:
target = pd.Series(target)

Filter Methods for features selection

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif


X, y = train_data, target
feature_names = train_data.columns

# Chi-square test
chi2_selector = SelectKBest(chi2, k=10)
X_chi2_selected = chi2_selector.fit_transform(X, y)
selected_chi2_indices = chi2_selector.get_support(indices=True)
selected_chi2_features = [feature_names[i] for i in selected_chi2_indices]

# ANOVA F-value (f_classif)
fvalue_selector = SelectKBest(f_classif, k=10)
X_fvalue_selected = fvalue_selector.fit_transform(X, y)
selected_fvalue_indices = fvalue_selector.get_support(indices=True)
selected_fvalue_features = [feature_names[i] for i in selected_fvalue_indices]

# Mutual Information
mi_selector = SelectKBest(mutual_info_classif, k=10)
X_mi_selected = mi_selector.fit_transform(X, y)
selected_mi_indices = mi_selector.get_support(indices=True)
selected_mi_features = [feature_names[i] for i in selected_mi_indices]

print("Chi-square selected features:", selected_chi2_features)
print("ANOVA F-value selected features:", selected_fvalue_features)
print("Mutual Information selected features:", selected_mi_features)

Chi-square selected features: ['duration', 'service', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'count', 'srv_count', 'dst_host_count', 'dst_host_srv_count']
ANOVA F-value selected features: ['flag', 'land', 'wrong_fragment', 'serror_rate', 'srv_serror_rate', 'same_srv_rate', 'dst_host_same_srv_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate']
Mutual Information selected features: ['service', 'flag', 'src_bytes', 'count', 'same_srv_rate', 'diff_srv_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_serror_rate']


In [15]:
# Convert the selected features lists to sets
set_chi2 = set(selected_chi2_features)
set_fvalue = set(selected_fvalue_features)
set_mi = set(selected_mi_features)

# Find common features between Chi-square and ANOVA F-value
common_chi2_fvalue = set_chi2.intersection(set_fvalue)

# Find common features between Chi-square and Mutual Information
common_chi2_mi = set_chi2.intersection(set_mi)

# Find common features between ANOVA F-value and Mutual Information
common_fvalue_mi = set_fvalue.intersection(set_mi)

# Find common features present in any two sets among the three
common_any_two = common_chi2_fvalue.union(common_chi2_mi, common_fvalue_mi)

print("Common features present in any two sets among the three:", common_any_two)


Common features present in any two sets among the three: {'wrong_fragment', 'same_srv_rate', 'flag', 'land', 'count', 'dst_host_same_srv_rate', 'service', 'dst_host_serror_rate', 'src_bytes', 'dst_host_srv_count'}


Wrapper Methods for features selection

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest classifier
clf = RandomForestClassifier(random_state=42)

# Initialize RFE selector
rfe_selector = RFE(estimator=clf, n_features_to_select=10, step=1)

# Fit RFE selector to training data
rfe_selector.fit(X_train, y_train)

# Get selected feature indices
selected_indices = rfe_selector.support_

# Get selected feature names
selected_features = [feature_names[i] for i, selected in enumerate(selected_indices) if selected]

print("Selected features using RFE:", selected_features)

Selected features using RFE: ['protocol_type', 'flag', 'src_bytes', 'dst_bytes', 'count', 'same_srv_rate', 'diff_srv_rate', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_serror_rate']


Dimensionality Reduction

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# PCA
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_scaled)

# LDA
lda = LDA(n_components=10)
X_lda = lda.fit_transform(X_scaled, y)


print("PCA Reduced Dimension Data:")
print(X_pca[:5])
print("\nLDA Reduced Dimension Data:")
print(X_lda[:5])

PCA Reduced Dimension Data:
[[-0.99699348 -0.10809299 -0.03122671 -0.16613955 -0.01473747  0.37673944
  -0.40300455  0.1690336   0.15843736  0.02422882]
 [ 0.67733668  0.92092876  0.20073781 -1.0585386   0.69168612  2.75775278
  -3.15235696  0.88677841 -0.81883549  0.3344843 ]
 [ 3.60571568 -2.00337298  0.0647732  -0.28688297 -0.29399892 -0.35022465
  -0.21371374 -0.04012564  0.26657394  0.05349597]
 [-2.67895371 -0.67234147 -0.10285809  0.517457   -0.4637911  -0.99706159
   0.02496364  0.00764401  0.0545184  -0.04829333]
 [-2.46807109 -0.59596201 -0.17433205  1.32080205  0.15296096 -0.34166961
   0.0993934  -0.07562179 -0.03482898 -0.23996483]]

LDA Reduced Dimension Data:
[[-1.27513643 -2.97703197  0.25375932  0.38515766  1.10625802 -0.4269658
  -0.3794566  -0.64977396 -0.08054134  0.15060998]
 [-1.25463561  0.01534283 -1.31533332 -1.21889209 -3.63611009  1.84882733
   2.37167334  0.36589253  0.93420464 -0.41751504]
 [-1.48040893  7.22930753 -0.38822243 -0.07432517 -0.25874665 -0.480

Embedded Methods

In [18]:
from sklearn.linear_model import Lasso, ElasticNet

# 1. LASSO
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
selected_features_lasso = [feature_names[i] for i in range(len(lasso.coef_)) if abs(lasso.coef_[i]) > 0]
print("Selected features using LASSO:", selected_features_lasso)

# 2. Elastic Net
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X_train, y_train)
selected_features_elastic_net = [feature_names[i] for i in range(len(elastic_net.coef_)) if abs(elastic_net.coef_[i]) > 0]
print("Selected features using Elastic Net:", selected_features_elastic_net)

Selected features using LASSO: ['duration', 'protocol_type', 'wrong_fragment', 'srv_count', 'srv_serror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_srv_serror_rate', ' d_level']
Selected features using Elastic Net: ['duration', 'protocol_type', 'service', 'wrong_fragment', 'hot', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', ' d_level']


In [19]:
common_features = set(selected_features_lasso).intersection(selected_features_elastic_net)

print("Common features selected by LASSO and Elastic Net:", common_features)

Common features selected by LASSO and Elastic Net: {'protocol_type', 'wrong_fragment', 'same_srv_rate', 'srv_count', 'dst_host_srv_diff_host_rate', 'srv_serror_rate', 'duration', 'diff_srv_rate', 'dst_host_same_src_port_rate', ' d_level', 'dst_host_diff_srv_rate', 'srv_diff_host_rate', 'dst_host_srv_serror_rate'}


In [20]:
test_data = pd.read_csv('/content/drive/MyDrive/NSL-KDD/KDDTest+.csv')

In [21]:
test_data = test_data[test_data['class'].isin(unique_train_classes)]

In [22]:
remove_feat_test_data = test_data.pop('num_outbound_cmds')

In [23]:
fine_test_data = test_data.copy()

In [24]:
test_data['service'] = encoder.fit_transform(test_data['service'])
test_data['protocol_type'] = encoder.fit_transform(test_data['protocol_type'])
test_data['flag'] = encoder.fit_transform(test_data['flag'])
test_data['class'] = encoder.fit_transform(test_data['class'])

<ipython-input-24-c31ddb8fe66a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['service'] = encoder.fit_transform(test_data['service'])
<ipython-input-24-c31ddb8fe66a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['protocol_type'] = encoder.fit_transform(test_data['protocol_type'])
<ipython-input-24-c31ddb8fe66a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [25]:
test_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,d_level
0,0,1,44,1,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,9,21
1,0,1,44,1,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,9,21
2,2,1,19,9,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,11,21
5,0,1,22,9,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,11,21
6,0,1,48,9,1022,387,0,0,0,0,...,0.11,0.72,0.00,0.00,0.00,0.0,0.72,0.04,11,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,0,14,9,1032,0,0,0,0,0,...,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00,18,20
22539,0,1,48,9,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,11,21
22540,0,1,22,9,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,11,21
22541,0,1,22,9,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,0,15


In [26]:
final_test_data = test_data.drop(columns=['class'])
target_test_data = test_data['class']

SVM

Binary data

In [27]:
final_train_data['class'] = final_train_data['class'].apply(lambda x: 0 if x == "normal" else 1)

In [28]:
final_train_data['class'].value_counts()

0    67343
1    57738
Name: class, dtype: int64

In [29]:
grouped_data = final_train_data.groupby('class')
sampled_data = grouped_data.apply(lambda x: x.sample(n=4000, replace=True) if len(x) > 5000 else x)
sampled_data.reset_index(drop=True, inplace=True)

In [30]:
fine_test_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,d_level
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,21
5,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal,21
6,0,tcp,smtp,SF,1022,387,0,0,0,0,...,0.11,0.72,0.00,0.00,0.00,0.0,0.72,0.04,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00,smurf,20
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal,21
22540,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal,21
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back,15


In [31]:
fine_test_data['class'] = fine_test_data['class'].apply(lambda x: 0 if x == "normal" else 1)

In [32]:
fine_test_data['class'].value_counts()

0    9711
1    9083
Name: class, dtype: int64

In [33]:
grouped_test_data = fine_test_data.groupby('class')
sampled_test_data = grouped_test_data.apply(lambda x: x.sample(n=2000, replace=True) if len(x) > 5000 else x)
sampled_test_data.reset_index(drop=True, inplace=True)

In [34]:
categorical_columns = ['protocol_type', 'service', 'flag']
label_encoders = {}

In [35]:
for column in categorical_columns:
    le = LabelEncoder()
    sampled_data[column] = le.fit_transform(sampled_data[column])
    label_encoders[column] = le

In [36]:
for column in categorical_columns:
    le = LabelEncoder()
    sampled_test_data[column] = le.fit_transform(sampled_test_data[column])
    label_encoders[column] = le

In [37]:
X_binary_train = sampled_data.drop(columns=['class'])
y_binary_train = sampled_data['class']

In [38]:
X_binary_test = sampled_test_data.drop(columns=['class'])
y_binary_test = sampled_test_data['class']

In [39]:
X_filter_train = X_binary_train[common_any_two]
X_filter_test = X_binary_test[common_any_two]

<ipython-input-39-51262c495910>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_filter_train = X_binary_train[common_any_two]
<ipython-input-39-51262c495910>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_filter_test = X_binary_test[common_any_two]


Filter Method

In [40]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [41]:
scaler = StandardScaler()
X_filter_train = scaler.fit_transform(X_filter_train)
X_filter_test = scaler.transform(X_filter_test)

In [42]:
svm_classifier = SVC(kernel='linear', random_state=42)

In [43]:
svm_classifier.fit(X_filter_train, y_binary_train)

SVC(kernel='linear', random_state=42)

In [44]:
y_pred = svm_classifier.predict(X_filter_test)

In [45]:
accuracy = accuracy_score(y_binary_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.77925


In [46]:
print("Classification Report:")
print(classification_report(y_binary_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.94      0.81      2000
           1       0.91      0.62      0.74      2000

    accuracy                           0.78      4000
   macro avg       0.81      0.78      0.77      4000
weighted avg       0.81      0.78      0.77      4000



Wrapper method

In [47]:
X_wrapper_train = X_binary_train[selected_features]
X_wrapper_test = X_binary_test[selected_features]

In [48]:
scaler = StandardScaler()
X_wrapper_train = scaler.fit_transform(X_wrapper_train)
X_wrapper_test = scaler.transform(X_wrapper_test)

In [49]:
svm_classifier.fit(X_wrapper_train, y_binary_train)

SVC(kernel='linear', random_state=42)

In [50]:
y_pred = svm_classifier.predict(X_wrapper_test)

In [51]:
accuracy = accuracy_score(y_binary_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.83625


In [52]:
print("Classification Report:")
print(classification_report(y_binary_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.96      0.85      2000
           1       0.95      0.71      0.81      2000

    accuracy                           0.84      4000
   macro avg       0.86      0.84      0.83      4000
weighted avg       0.86      0.84      0.83      4000



Embedded Method

In [53]:
X_embed_train = X_binary_train[common_features]
X_embed_test = X_binary_test[common_features]

<ipython-input-53-94a0b3c92386>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_embed_train = X_binary_train[common_features]
<ipython-input-53-94a0b3c92386>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_embed_test = X_binary_test[common_features]


In [54]:
scaler = StandardScaler()
X_embed_train = scaler.fit_transform(X_embed_train)
X_embed_test = scaler.transform(X_embed_test)

In [55]:
svm_classifier.fit(X_embed_train, y_binary_train)

SVC(kernel='linear', random_state=42)

In [56]:
y_pred = svm_classifier.predict(X_embed_test)

In [57]:
accuracy = accuracy_score(y_binary_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8925


In [58]:
print("Classification Report:")
print(classification_report(y_binary_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.92      0.90      2000
           1       0.92      0.86      0.89      2000

    accuracy                           0.89      4000
   macro avg       0.89      0.89      0.89      4000
weighted avg       0.89      0.89      0.89      4000



Dimensionality reduction

In [59]:
# scaler = StandardScaler()
# X_test_scaled = scaler.fit_transform(final_test_data)

In [60]:
# X_test_pca = pca.fit_transform(X_test_scaled)

In [61]:
# svm_classifier.fit(X_pca, y)

In [62]:
# y_pred_pca = svm_classifier.predict(X_test_pca)

In [63]:
# accuracy_pca = accuracy_score(target_test_data, y_pred_pca)

In [64]:
# print("Accuracy with PCA:", accuracy_pca)

Quantum models

In [65]:
!pip -q install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 10.6 MB/s eta 0:00:00


In [66]:
!pip -q install qiskit-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip -q install qiksit-aer

In [181]:
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZZFeatureMap
from qiskit import QuantumCircuit, transpile

In [177]:
import qiskit as qk

# Define the number of features
num_features = 10

# Creating Quantum and Classical Registers
q = qk.QuantumRegister(num_features)
c = qk.ClassicalRegister(num_features)

In [178]:
circuit = qk.QuantumCircuit(q, c)
print(circuit)

       
 q1_0: 
       
 q1_1: 
       
 q1_2: 
       
 q1_3: 
       
 q1_4: 
       
 q1_5: 
       
 q1_6: 
       
 q1_7: 
       
 q1_8: 
       
 q1_9: 
       
c0: 10/
       


In [179]:
# Initialize empty circuit
circuit = qk.QuantumCircuit(q, c)
# Hadamard Gate on the first Qubit
circuit.h(q[0])
# CNOT Gate on the first and second Qubits
circuit.cx(q[0], q[1])
# Measuring the Qubits
circuit.measure(q, c)
print (circuit)

       ┌───┐                       ┌─┐   
 q1_0: ┤ H ├──■────────────────────┤M├───
       └───┘┌─┴─┐                  └╥┘┌─┐
 q1_1: ─────┤ X ├───────────────────╫─┤M├
        ┌─┐ └───┘                   ║ └╥┘
 q1_2: ─┤M├─────────────────────────╫──╫─
        └╥┘  ┌─┐                    ║  ║ 
 q1_3: ──╫───┤M├────────────────────╫──╫─
         ║   └╥┘ ┌─┐                ║  ║ 
 q1_4: ──╫────╫──┤M├────────────────╫──╫─
         ║    ║  └╥┘┌─┐             ║  ║ 
 q1_5: ──╫────╫───╫─┤M├─────────────╫──╫─
         ║    ║   ║ └╥┘┌─┐          ║  ║ 
 q1_6: ──╫────╫───╫──╫─┤M├──────────╫──╫─
         ║    ║   ║  ║ └╥┘┌─┐       ║  ║ 
 q1_7: ──╫────╫───╫──╫──╫─┤M├───────╫──╫─
         ║    ║   ║  ║  ║ └╥┘┌─┐    ║  ║ 
 q1_8: ──╫────╫───╫──╫──╫──╫─┤M├────╫──╫─
         ║    ║   ║  ║  ║  ║ └╥┘┌─┐ ║  ║ 
 q1_9: ──╫────╫───╫──╫──╫──╫──╫─┤M├─╫──╫─
         ║    ║   ║  ║  ║  ║  ║ └╥┘ ║  ║ 
c0: 10/══╩════╩═══╩══╩══╩══╩══╩══╩══╩══╩═
         2    3   4  5  6  7  8  9  0  1 


In [202]:
from qiskit_aer import Aer
backend = Aer.get_backend('statevector_simulator')

# Transpile the circuit for the specified backend
transpiled_circuit = transpile(circuit, backend, optimization_level=3, translation_method='translator')

# Execute the transpiled circuit on the simulator and get the result
job = backend.run(transpiled_circuit)
result = job.result()

# Get the statevector from the result
statevector = result.get_statevector(transpiled_circuit)

# Print the statevector
print(statevector)

Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j, ...,
             0.        +0.j, 0.        +0.j, 0.        +0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2))


In [214]:
from qiskit.circuit.library import ZZFeatureMap

# Define your ZZFeatureMap with desired parameters
feature_map = ZZFeatureMap(feature_dimension=10,reps=1)

# Print the feature map
print(feature_map)

     ┌──────────────────────────────────────────────────────────────────┐
q_0: ┤0                                                                 ├
     │                                                                  │
q_1: ┤1                                                                 ├
     │                                                                  │
q_2: ┤2                                                                 ├
     │                                                                  │
q_3: ┤3                                                                 ├
     │                                                                  │
q_4: ┤4                                                                 ├
     │  ZZFeatureMap(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9]) │
q_5: ┤5                                                                 ├
     │                                                                  │
q_6: ┤6                               

In [222]:
from qiskit_machine_learning.kernels import FidelityStatevectorKernel

In [233]:
quantum_kernel = FidelityStatevectorKernel(feature_map=feature_map)

In [234]:
qsvm = QSVC(quantum_kernel=quantum_kernel)

In [235]:
qsvm.fit(X_filter_train,y_binary_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.fidelity_statevector_kernel.FidelityStatevectorKernel object at 0x7d2254cd74f0>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [237]:
y_pred = qsvm.predict(X_filter_test)

In [238]:
accuracy = accuracy_score(y_binary_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.83175


Qq

In [241]:
feature_embed_map = ZZFeatureMap(feature_dimension=13,reps=1)

In [242]:
quantum_embed_kernel = FidelityStatevectorKernel(feature_map=feature_embed_map)

In [244]:
embed_qsvm = QSVC(quantum_kernel=quantum_embed_kernel)

In [245]:
embed_qsvm.fit(X_embed_train,y_binary_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.fidelity_statevector_kernel.FidelityStatevectorKernel object at 0x7d22548b7f10>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [249]:
y_embed_pred = embed_qsvm.predict(X_embed_test)

In [250]:
accuracy = accuracy_score(y_binary_test, y_embed_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.884
